In [1]:
!pip install -q condacolab
import condacolab

condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:08
🔁 Restarting kernel...


In [2]:
!mamba install -y -c conda-forge rdkit openmm openff-toolkit


Looking for: ['rdkit', 'openmm']

[+] 0.0s
conda-forge/linux-64  ⣾  
conda-forge/noarch    ⣾  [+] 0.1s
conda-forge/linux-64   1%
conda-forge/noarch    ⣾  [+] 0.2s
conda-forge/linux-64  16%
conda-forge/noarch    18%[+] 0.3s
conda-forge/linux-64  24%
conda-forge/noarch    54%[+] 0.4s
conda-forge/linux-64  31%
conda-forge/noarch    72%[+] 0.5s
conda-forge/linux-64  47%
conda-forge/noarch   100%[+] 0.6s
conda-forge/linux-64  47%
conda-forge/noarch   100%conda-forge/noarch                                
[+] 0.7s
conda-forge/linux-64  63%[+] 0.8s
conda-forge/linux-64  74%[+] 0.9s
conda-forge/linux-64  96%[+] 1.0s
conda-forge/linux-64 100%conda-forge/linux-64                              

Pinned packages:
  - python 3.10.*
  - python 3.10.*
  - python_abi 3.10.* *cp310*
  - cuda-version 12.*


Transaction

  Prefix: /usr/local

  Updating specs:

   - rdkit
   - openmm
   - ca-certificates
   - certifi
   - openssl


  Package                          Version  Build                Channel 

In [2]:
!git clone https://github.com/hodakamori/ml-tutorial.git

Cloning into 'ml-tutorial'...
remote: Enumerating objects: 609, done.
remote: Counting objects: 100% (302/302), done.
remote: Compressing objects: 100% (240/240), done.
remote: Total 609 (delta 90), reused 211 (delta 49), pack-reused 307 (from 2)
Receiving objects: 100% (609/609), 92.23 MiB | 12.44 MiB/s, done.
Resolving deltas: 100% (192/192), done.
Updating files: 100% (142/142), done.


In [3]:
!cp /content/ml-tutorial/042_coarse_grained_modelling/*.py .

In [4]:
import sys

from forcefield import CGForceField
from model_builder import CGModelBuilder
from openmm import LangevinMiddleIntegrator, MonteCarloBarostat, unit
from openmm.app import PDBFile, Simulation, StateDataReporter, XTCReporter


In [13]:
builder = CGModelBuilder(
    smiles_sequence="[EO]100",
    bond_length=0.36,  # nm
    min_dist=0.41,  # nm
    max_tries=50000,
)
topology, positions = builder.create_molecule()

with open("peo100.pdb", "w") as f:
    PDBFile.writeFile(topology, positions, f)

structures = [
    {
        "pdb": "peo100.pdb",
        "number": 1000,
        "box": (0, 0, 0, 250, 250, 250),
    },
]
topology, positions = builder.create_packed_model(
    structures,
    box_size=(250, 250, 250),  # nm
    output_pdb="peo10_100.pdb",
)

In [14]:
param_dict = {
    "BOND": {
        ("EO", "EO"): {"r0": 0.36, "k": 7000},
    },
    "ANGLE": {
        ("EO", "EO", "EO"): {"theta0": 2.147, "k": 80},
    },
    "TORSION": {
        ("EO", "EO", "EO", "EO"): [
            {"periodicity": 1, "phase": 0.0, "k": -1.328},
            {"periodicity": 2, "phase": 0.0, "k": -0.437},
            {"periodicity": 3, "phase": 0.0, "k": 1.081},
            {"periodicity": 4, "phase": 0.0, "k": 0.680},
        ]
    },
    "NONBONDED": {
        "EO": {"sigma": 0.41, "epsilon": 2.35, "charge": 0.0, "mass": 44.0},
    },
}
ff = CGForceField.from_dict(param_dict)
system = ff.create_system(topology)

In [ ]:
temperature = 300 * unit.kelvin
friction = 10 / unit.picosecond
timestep = 20 * unit.femtosecond
integrator = LangevinMiddleIntegrator(temperature, friction, timestep)
simulation = Simulation(topology, system, integrator)

barostat = MonteCarloBarostat(1.0 * unit.atmospheres, temperature)
simulation.system.addForce(barostat)

simulation.context.setPositions(positions)

simulation.reporters.append(XTCReporter("md_traj.xtc", 1000))
simulation.reporters.append(
    StateDataReporter(
        sys.stdout,
        1000,
        step=True,
        time=True,
        potentialEnergy=True,
        kineticEnergy=True,
        temperature=True,
        volume=True,
        density=True,
    )
)
simulation.reporters.append(
    StateDataReporter(
        "md.log",
        1000,
        step=True,
        time=True,
        potentialEnergy=True,
        kineticEnergy=True,
        temperature=True,
        volume=True,
        density=True,
    )
)

state = simulation.context.getState(
    getPositions=True, getEnergy=True, enforcePeriodicBox=True
)
PDBFile.writeFile(
    simulation.topology, state.getPositions(), open("md_traj_init.pdb", "w")
)
simulation.context.reinitialize(preserveState=True)
simulation.minimizeEnergy()
simulation.context.setVelocitiesToTemperature(300 * unit.kelvin)
simulation.step(10_000_000)

#"Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Temperature (K)","Box Volume (nm^3)","Density (g/mL)"
1000,39.999999999999325,-835515.2767863498,374660.2246589728,300.40845040380987,11127248.119213626,0.000656619841427718
2000,80.00000000000131,-948436.9212109213,373951.270802608,299.8400000709991,7983367.947774253,0.0009151991920404879
3000,120.00000000000756,-1000497.4614931371,374577.85154845513,300.3424023505155,6914599.982170352,0.001056658651896679
4000,160.00000000000244,-1030707.3909828364,374900.6242454732,300.60120656660786,6371016.423967396,0.0011468141673719088
5000,199.9999999999945,-1049071.733076784,373358.3926832299,299.36462108652637,6010359.573018552,0.001215629748403802
6000,239.99999999998653,-1060115.5756144896,373959.06130034593,299.8462466143984,5752973.518352878,0.001270016604849034
7000,279.9999999999956,-1070646.7144777393,374560.9501280469,300.3288505262751,5512392.989214226,0.00132544466801639
8000,320.0000000000161,-1078982.0848